In [143]:
import os
import artm
from pymongo import MongoClient
import faiss
from nltk.stem.snowball import SnowballStemmer
from pymystem3 import Mystem
import logging
from nltk.tokenize import sent_tokenize
import bs4
import re
import uuid
import numpy as np

In [99]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [91]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
models_path = os.path.join("..", "..", "data", "models")
experiments_path = os.path.join("..", "..", "experiments")
external_data_path = os.path.join("..", "..", "data", "external")

In [50]:
# Посмотреть на темы, найденные моделью

In [52]:
model_name = "e8e6ca46-70c7-11ea-93cd-bbb79b83363b"

In [54]:
%%time

model = artm.load_artm_model(os.path.join(models_path, model_name))

CPU times: user 6.15 s, sys: 2.51 s, total: 8.65 s
Wall time: 4.39 s


In [61]:
model.score_tracker

{'PerplexityScore': <artm.score_tracker.PerplexityScoreTracker at 0x7f14939918d0>,
 'SparsityPhiScore': <artm.score_tracker.SparsityPhiScoreTracker at 0x7f1492fe0590>,
 'SparsityThetaScore': <artm.score_tracker.SparsityThetaScoreTracker at 0x7f1492fe04d0>,
 'TopTokensScoreText': <artm.score_tracker.TopTokensScoreTracker at 0x7f1492fe0550>,
 'TopicKernelScore': <artm.score_tracker.TopicKernelScoreTracker at 0x7f1492fe09d0>,
 'ITMScore_f1fd572b-32b7-423f-a9ad-f4901e48a69d': <artm.score_tracker.TopicMassPhiScoreTracker at 0x7f1492fe0910>}

In [64]:
for topic_name in model.topic_names[:380]:
    print(topic_name + ': ')
    print([token.split("_")[0] for token in model.score_tracker['TopTokensScoreText'].last_tokens[topic_name]], "\n\n")

objective_topic_0: 
['встреча', 'переговоры', 'договариваться', 'диалог', 'визит', 'мид', 'нга', 'саммит', 'договоренность', 'делегация', 'посол', 'урегулирование', 'дипломат', 'дипломатический', 'двусторонний', 'посольство', 'лавров', 'миссия', 'молдавия', 'компромисс'] 


objective_topic_1: 
['международный', 'соглашение', 'санкция', 'иран', 'оон', 'конвенция', 'санкционный', 'иранский', 'резолюция', 'тегеран', 'антироссийский', 'трансграничный', 'многосторонний', 'ратифицировать', 'ратификация', 'межгосударственный', 'суверенный', 'юрисдикция', 'хартия', 'присоединение'] 


objective_topic_2: 
['выходить', 'правда', 'вернуться', 'приносить', 'сильно', 'плохо', 'заканчиваться', 'долгий', 'держаться', 'федор', 'извинение', 'хабиб', 'смешанный', 'ufc', 'макгрегор', 'гонорар', 'наверняка', 'единоборство', 'емельяненко', 'конор'] 


objective_topic_3: 
['информация', 'сообщение', 'портал', 'блокировка', 'роскомнадзор', 'раскрытие', 'заблокировать', 'блокировать', 'конфиденциальный', 'инф

In [27]:
# Достать из БД текст по отступу

In [66]:
client = MongoClient('localhost', 27017)
db = client.publicru_test
collection = db.documents_collection

In [67]:
n = 15
# result = collection.find({}).skip(0 if n == 0 else (n - 1)).limit(1)
result = collection.find({}).skip(n).limit(1)
result[0]["title"]

'Легкий Jaguar'

In [68]:
limit = 20
i = 0
for doc in collection.find({}):
    print(i, doc["title"])
    if i == limit:
        break
    i += 1

0 Эффект слабого рубля
1 РОССТАТ
2 Неотложка с планшетом
3 ПРОГНОЗЫ НОМЕРА
4 ТАНЕЦ НЕНАСТОЯЩЕГО ЛЕБЕДЯ
5 Поездам фиолетово
6 ОБНУЛЕНИЕ ПРОШЛОГО
7 ВРЕМЯ БОЛЬШИХ ВОЗМОЖНОСТЕЙ
8 С долей лояльности
9 Связной банк: конец истории
10 ВЕРНОЕ РЕШЕНИЕ
11 Черный код за углом
12 Прогресс возможен
13 БРАСС ВСЕ ЕЩЕ БЫСТРЫЙ
14 Деловой завтрак
15 Легкий Jaguar
16 Женщины в бизнесе
17 ОСОБОЕ ПРИГЛАШЕНИЕ
18 МЕСТА ЗНАТЬ НАДО
19 В ГОРОД ВОЗВРАЩАЕТСЯ ОБЫЧНАЯ ЖИЗНЬ
20 Эрдоган сменил не верных


In [28]:
# Преобразовать текст в эмбеденги

In [95]:
n = 15
# result = collection.find({}).skip(0 if n == 0 else (n - 1)).limit(1)
result = collection.find({}).skip(n).limit(1)
document = result[0]["body"]

In [96]:
document

'<body><p>В основе нового F-PACE - облегченная алюминиевая платформа Jaguar. Это единственный автомобиль с подобным кузовом в своем сегменте. Все версии F-PACE в России будут оснащены системой полного привода и автоматической трансмиссией.</p>\n<p><b>Бонусы</b></p>\n<p>Технология Activity Key: браслет с защитой от воды и ударов с интегрированным передатчиком</p>\n<p>Стереокамера - основа системы автоматического экстренного торможения</p>\n<p>Лазерный дисплей, который проецирует в поле зрения водителя всю информацию о поездке</p>\n<p>Мощность двигателя 180 л.с.</p>\n<p>Разгон 0-100 км/ч 8,7 с</p>\n<p>Максимальная скорость 208 км/ч</p>\n<p><b>Цена: от 3 193 000 руб.</b></p>\n<p><img alt="" src="226/1.jpg" title=""/></p>\n</body>'

In [111]:
def paras(post):
    """
    Генератор тараграфов
    """
    paras_tags = ["h1", "h2", "h3", "h4", "h5", "h6", "p", "li"]
    try:
        soup = bs4.BeautifulSoup(post, "lxml")
        for element in soup.find_all(paras_tags):
            yield element.text
        soup.decompose()
    except:
        logging.info(f"paras soup exception on text: {post}.")
        yield ""

In [112]:
def sents(post):
    """
    Генератор предложений
    """
    for para in paras(post):
        for sentence in sent_tokenize(para):
            yield sentence

In [113]:
def load_stop_words():
    """
    Загрузка стоп слов
    """
    with open(os.path.join(external_data_path, 'russian_stop_words.txt'), 'r') as handle:
        stop_words_ru = [line.rstrip('\n') for line in handle]
    with open(os.path.join(external_data_path, 'english_stop_words.txt'), 'r') as handle:
        stop_words_en = [line.rstrip('\n') for line in handle]
    return set(stop_words_ru + stop_words_en)

In [114]:
def lemmatize(text, morphological_analyzer, stemmer, stop_words):
    lemmatizing_sents = []
    for para in paras(text):
        for sentence in sents(para):
            lemmatizing_words = []
            for token_analize in morphological_analyzer.analyze(sentence):
                if "analysis" in token_analize:
                    if token_analize["text"] in stop_words:
                        continue
                    if token_analize["analysis"]:
                        find = re.compile("([^,=]+)")
                        lexem = token_analize["analysis"][0]["lex"]
                        if lexem in stop_words:
                            continue
                        grammemes = token_analize["analysis"][0]["gr"]
                        grammeme = re.search(find, grammemes).group(0)
                        lemmatizing_words.append(f"{lexem}_{grammeme}")
                    else:
                        lexem = stemmer.stem(token_analize["text"])
                        if lexem in stop_words:
                            continue
                        lemmatizing_words.append(lexem)
            lemmatizing_sents.append(lemmatizing_words)
    return lemmatizing_sents

In [115]:
morphological_analyzer = Mystem()
stemmer = SnowballStemmer("english")
stop_words = load_stop_words()

In [119]:
%%time

lemmatize(document, morphological_analyzer, stemmer, stop_words)

CPU times: user 20.1 ms, sys: 2.23 ms, total: 22.4 ms
Wall time: 17.8 ms


[['основа_S',
  'новый_A',
  'pace',
  'облегченный_A',
  'алюминиевый_A',
  'платформа_S',
  'jaguar'],
 ['единственный_A',
  'автомобиль_S',
  'подобный_A',
  'кузов_S',
  'свой_APRO',
  'сегмент_S'],
 ['версия_S',
  'pace',
  'россия_S',
  'оснащать_V',
  'система_S',
  'полный_A',
  'привод_S',
  'автоматический_A',
  'трансмиссия_S'],
 ['бонус_S'],
 ['технология_S',
  'activ',
  'key',
  'браслет_S',
  'защита_S',
  'вода_S',
  'удар_S',
  'интегрировать_V',
  'передатчик_S'],
 ['стереокамера_S',
  'основа_S',
  'система_S',
  'автоматический_A',
  'экстренный_A',
  'торможение_S'],
 ['лазерный_A',
  'дисплей_S',
  'проецировать_V',
  'поле_S',
  'зрение_S',
  'водитель_S',
  'информация_S',
  'поездка_S'],
 ['мощность_S', 'двигатель_S', 'л_S'],
 ['разгон_S', 'км_S', 'ч_S'],
 ['максимальный_A', 'скорость_S', 'км_S', 'ч_S'],
 ['цена_S']]

In [130]:
def create_batch_vectorizer(model, p_document):
    document = []
    for sentence in p_document:
        document += sentence
    vocab = list(set(document))
    batch = artm.messages.Batch()
    batch.id = str(uuid.uuid4())
    dictionary = {}
    use_bag_of_words = True

    for i, token in enumerate(vocab):
        batch.token.append(token)
        dictionary[token] = i

    item = batch.item.add()

    if use_bag_of_words:
        local_dict = {}
        for token in document:
            if not token in local_dict:
                local_dict[token] = 0
            local_dict[token] += 1

        for k, v in local_dict.items():
            item.token_id.append(dictionary[k])
            item.token_weight.append(v)
    else:
        for token in document:
            item.token_id.append(dictionary[token])
            item.token_weight.append(1.0)

    batch_vectorizer = artm.BatchVectorizer(
        batches=[batch],
        data_format='batches',
        process_in_memory_model=model,
        batch_name_type='guid',
    )
    return batch_vectorizer

In [131]:
def prepare_text(model, document):
    morphological_analyzer = Mystem()
    stemmer = SnowballStemmer("english")
    stop_words = load_stop_words()
    t_document = lemmatize(document, morphological_analyzer, stemmer, stop_words)
    batch_vectorizer = create_batch_vectorizer(model, t_document)
    theta = model.transform(batch_vectorizer=batch_vectorizer)
    embeddings = theta.T
    embeddings = embeddings.loc[:, "objective_topic_0": "objective_topic_379"]
    return embeddings

In [133]:
%%time

embeddings = prepare_text(model, document)

CPU times: user 212 ms, sys: 309 ms, total: 521 ms
Wall time: 843 ms


In [134]:
embeddings

,objective_topic_0,objective_topic_1,objective_topic_2,objective_topic_3,objective_topic_4,objective_topic_5,objective_topic_6,objective_topic_7,objective_topic_8,objective_topic_9,...,objective_topic_370,objective_topic_371,objective_topic_372,objective_topic_373,objective_topic_374,objective_topic_375,objective_topic_376,objective_topic_377,objective_topic_378,objective_topic_379
0,0.0,0.0,0.0,0.006132,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Найти значимые темы для документа

In [ ]:
for topic_name in model.topic_names[:380]:
    print(topic_name + ': ')
    print([token.split("_")[0] for token in model.score_tracker['TopTokensScoreText'].last_tokens[topic_name]], "\n\n")

In [137]:
def print_top_topic_words(model, topic_names=None):
    for topic_name in topic_names:
        print(topic_name + ': ')
        print(model.score_tracker['TopTokensScoreText'].last_tokens[topic_name], "\n\n")

In [138]:
print_top_topic_words(model, topic_names=embeddings.loc[0].sort_values(ascending=False)[:3].index)

objective_topic_28: 
['колесо_S', 'салон_S', 'задний_A', 'мотор_S', 'коробка_S', 'кузов_S', 'сидение_S', 'передний_A', 'руль_S', 'привод_S', 'кроссовер_S', 'подвеска_S', 'комплектация_S', 'автомат_S', 'кабина_S', 'внедорожник_S', 'багажник_S', 'седан_S', 'бензиновый_A', 'audi'] 


objective_topic_64: 
['ч_S', 'скорость_S', 'км_S', 'аппарат_S', 'максимальный_A', 'высота_S', 'расстояние_S', 'мина_S', 'беспилотный_A', 'беспилотник_S', 'дрон_S', 'летательный_A', 'скоростной_A', 'дронов_S', 'бпло_S', 'аэродинамический_A', 'бла_S', 'радиус_S', 'дирижабль_S', 'квадрокоптер_S'] 


objective_topic_340: 
['камера_S', 'автоматический_A', 'модуль_S', 'изображение_S', 'датчик_S', 'видеонаблюдение_S', 'ручной_A', 'распознавание_S', 'оптический_A', 'видеокамера_S', 'лазерный_A', 'монитор_S', 'отпечаток_S', 'встроенный_A', 'оснащать_V', 'сенсор_S', 'обнаружение_S', 'сканирование_S', 'дисплей_S', 'визуальный_A'] 




In [29]:
# Найти 10 ближайших текстов

In [139]:
index_name = "faiss_index_publicru-dataset-2020-03-27_e8e6ca46-70c7-11ea-93cd-bbb79b83363b.idx"

In [140]:
%%time

index = faiss.read_index(os.path.join(models_path, index_name))

CPU times: user 133 ms, sys: 58.3 ms, total: 191 ms
Wall time: 128 ms


In [159]:
def find(model, document):
    embeddings = prepare_text(model, document)
    print_top_topic_words(model, topic_names=embeddings.loc[0].sort_values(ascending=False)[:3].index)
    values = np.ascontiguousarray(embeddings.values)
    values = values.astype('float32')
    index = faiss.read_index(os.path.join(models_path, index_name))
    k = 10
    D, I = index.search(values[:1], k)
    return D[0], I[0]

In [144]:
%%time

dist, ind = find(model, document)

CPU times: user 546 ms, sys: 301 ms, total: 847 ms
Wall time: 999 ms


In [145]:
dist, ind

(array([0.00151771, 0.00921459, 0.01173045, 0.01323079, 0.01349461,
        0.01360025, 0.01435183, 0.01443191, 0.01510552, 0.01523606],
       dtype=float32),
 array([    15, 242786, 519808, 192472,  16199, 370712, 370694,  40337,
        337152, 262416]))

In [152]:
print("Результат поиска по тексту:\n")
print_top_topic_words(model, topic_names=embeddings.loc[0].sort_values(ascending=False)[:3].index)
for j, i in enumerate(ind):
    result = collection.find({}).skip(int(i)).limit(1)
    print("#", i, ":", result[0]["title"])
    print("distance: ", dist[j])
    print(result[0]["annotation"])
    print("\n\n")

Результат поиска по тексту:

objective_topic_28: 
['колесо_S', 'салон_S', 'задний_A', 'мотор_S', 'коробка_S', 'кузов_S', 'сидение_S', 'передний_A', 'руль_S', 'привод_S', 'кроссовер_S', 'подвеска_S', 'комплектация_S', 'автомат_S', 'кабина_S', 'внедорожник_S', 'багажник_S', 'седан_S', 'бензиновый_A', 'audi'] 


objective_topic_64: 
['ч_S', 'скорость_S', 'км_S', 'аппарат_S', 'максимальный_A', 'высота_S', 'расстояние_S', 'мина_S', 'беспилотный_A', 'беспилотник_S', 'дрон_S', 'летательный_A', 'скоростной_A', 'дронов_S', 'бпло_S', 'аэродинамический_A', 'бла_S', 'радиус_S', 'дирижабль_S', 'квадрокоптер_S'] 


objective_topic_340: 
['камера_S', 'автоматический_A', 'модуль_S', 'изображение_S', 'датчик_S', 'видеонаблюдение_S', 'ручной_A', 'распознавание_S', 'оптический_A', 'видеокамера_S', 'лазерный_A', 'монитор_S', 'отпечаток_S', 'встроенный_A', 'оснащать_V', 'сенсор_S', 'обнаружение_S', 'сканирование_S', 'дисплей_S', 'визуальный_A'] 


# 15 : Легкий Jaguar
distance:  0.0015177074
В основе новог

In [155]:
def search(doc_id, model):
    result = collection.find({}).skip(doc_id).limit(1)
    print(result[0]["annotation"])
    document = result[0]["body"]

    dist, ind = find(model, document)

    print("Результат поиска по тексту:\n")
    print_top_topic_words(model, topic_names=embeddings.loc[0].sort_values(ascending=False)[:3].index)
    for j, i in enumerate(ind):
        result = collection.find({}).skip(int(i)).limit(1)
        print("#", i, ":", result[0]["title"])
        print("distance: ", dist[j])
        print(result[0]["annotation"])
        print("\n\n")


In [156]:
search(262416, model)

Год выпуска - 2015. Редакционная Веста продолжает досаждать мелкими неисправностями. Из последних - заживший своей жизнью экран штатной магнитолы.
Результат поиска по тексту:

objective_topic_28: 
['колесо_S', 'салон_S', 'задний_A', 'мотор_S', 'коробка_S', 'кузов_S', 'сидение_S', 'передний_A', 'руль_S', 'привод_S', 'кроссовер_S', 'подвеска_S', 'комплектация_S', 'автомат_S', 'кабина_S', 'внедорожник_S', 'багажник_S', 'седан_S', 'бензиновый_A', 'audi'] 


objective_topic_64: 
['ч_S', 'скорость_S', 'км_S', 'аппарат_S', 'максимальный_A', 'высота_S', 'расстояние_S', 'мина_S', 'беспилотный_A', 'беспилотник_S', 'дрон_S', 'летательный_A', 'скоростной_A', 'дронов_S', 'бпло_S', 'аэродинамический_A', 'бла_S', 'радиус_S', 'дирижабль_S', 'квадрокоптер_S'] 


objective_topic_340: 
['камера_S', 'автоматический_A', 'модуль_S', 'изображение_S', 'датчик_S', 'видеонаблюдение_S', 'ручной_A', 'распознавание_S', 'оптический_A', 'видеокамера_S', 'лазерный_A', 'монитор_S', 'отпечаток_S', 'встроенный_A', 'осна

In [160]:
document = """
Маски могут иметь разную конструкцию. Они могут быть одноразовыми или могут применяться многократно. Есть маски, которые служат 2, 4, 6 часов. Стоимость этих масок различная, из-за различной пропитки. Но нельзя все время носить одну и ту же маску, тем самым вы можете инфицировать дважды сами себя. Какой стороной внутрь носить медицинскую маску — непринципиально.

Чтобы обезопасить себя от заражения, крайне важно правильно ее носить:

обычную медицинскую маску необходимо менять каждые 2 часа
маска должна тщательно закрепляться, плотно закрывать рот и нос, не оставляя зазоров;
старайтесь не касаться поверхностей маски при ее снятии, если вы ее коснулись, тщательно вымойте руки с мылом или спиртовым средством;
 влажную или отсыревшую маску следует сменить на новую, сухую;
не используйте вторично одноразовую маску;
использованную одноразовую маску следует немедленно выбросить в отходы.
При уходе за больным, после окончания контакта с заболевшим, маску следует немедленно снять. После снятия маски необходимо незамедлительно и тщательно вымыть руки.

Маска уместна, если вы находитесь в месте массового скопления людей, в общественном транспорте, а также при уходе за больным, но она нецелесообразна на открытом воздухе.

Во время пребывания на улице полезно дышать свежим воздухом и маску надевать не стоит. Вместе с тем, медики напоминают, что эта одиночная мера не обеспечивает полной защиты от заболевания. Кроме ношения маски необходимо соблюдать другие профилактические меры.
"""

dist, ind = find(model, document)

print("Результат поиска по тексту:\n")
for j, i in enumerate(ind):
    result = collection.find({}).skip(int(i)).limit(1)
    print("#", i, ":", result[0]["title"])
    print("distance: ", dist[j])
    print(result[0]["annotation"])
    print("\n\n")

objective_topic_187: 
['вирус_S', 'инфекция_S', 'санитарный_A', 'коронавирус_S', 'роспотребнадзор_S', 'грипп_S', 'эпидемия_S', 'профилактика_S', 'попов_S', 'вакцина_S', 'вич_A', 'эпидемиологический_A', 'заболевать_V', 'вирусный_A', 'прививка_S', 'инфекционный_A', 'карантин_S', 'иммунитет_S', 'заражение_S', 'вспышка_S'] 


objective_topic_245: 
['растение_S', 'цветок_S', 'кофе_S', 'запах_S', 'трава_S', 'уборка_S', 'дача_S', 'сорт_S', 'роза_S', 'баня_S', 'посуда_S', 'выращивать_V', 'ягода_S', 'огород_S', 'чашка_S', 'плод_S', 'букет_S', 'корень_S', 'мед_S', 'семя_S'] 


objective_topic_301: 
['получать_V', 'необходимо_ADV', 'направлять_V', 'должный_A', 'предварительно_ADV', 'распределять_V', 'направленный_A'] 


Результат поиска по тексту:

objective_topic_28: 
['колесо_S', 'салон_S', 'задний_A', 'мотор_S', 'коробка_S', 'кузов_S', 'сидение_S', 'передний_A', 'руль_S', 'привод_S', 'кроссовер_S', 'подвеска_S', 'комплектация_S', 'автомат_S', 'кабина_S', 'внедорожник_S', 'багажник_S', 'седан_S